<a href="https://colab.research.google.com/github/isaacdono/ml-studies/blob/main/rnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Estudo Prático sobre Redes Neurais Recorrentes (RNNs)

Bem-vindo a este guia prático sobre RNNs!

As **Redes Neurais Recorrentes** são especializadas em processar **dados sequenciais**, como séries temporais, texto ou áudio. Sua principal característica é possuir uma "memória" interna (um loop), que permite que informações de passos anteriores persistam e influenciem as saídas futuras.

Neste notebook, vamos:
1.  **Gerar Dados:** Criar uma série temporal simples (uma onda senoidal).
2.  **Pré-processar os Dados:** Preparar os dados para o formato que a RNN espera.
3.  **Construir uma RNN Simples:** Usaremos `SimpleRNN` do Keras para prever o próximo ponto da sequência.
4.  **Construir uma LSTM:** Implementaremos uma `LSTM` (Long Short-Term Memory), uma RNN mais avançada que lida melhor com dependências de longo prazo.
5.  **Avaliar e Comparar:** Visualizaremos as previsões de ambos os modelos para entender seu desempenho.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, LSTM, Dense
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

print(f"TensorFlow versão: {tf.__version__}")
print("Bibliotecas importadas com sucesso!")


In [ ]:
# Gerando uma onda senoidal
t = np.arange(0, 150, 0.1)
amplitude = np.sin(t) + np.sin(t*0.5) # Adicionando complexidade

# Visualizando os dados
plt.figure(figsize=(12, 5))
plt.plot(t, amplitude)
plt.title("Série Temporal: Onda Senoidal")
plt.xlabel("Tempo")
plt.ylabel("Amplitude")
plt.grid(True)
plt.show()

In [ ]:
# Dividindo em treino e teste
train_size = int(len(data_scaled) * 0.70)
test_size = len(data_scaled) - train_size
train_data, test_data = data_scaled[0:train_size, :], data_scaled[train_size:len(data_scaled), :]

# Função para criar sequências
# A ideia é usar 'n' passos anteriores (look_back) para prever o próximo passo
def create_sequences(dataset, look_back=10):
    X, Y = [], []
    for i in range(len(dataset) - look_back - 1):
        a = dataset[i:(i + look_back), 0]
        X.append(a)
        Y.append(dataset[i + look_back, 0])
    return np.array(X), np.array(Y)

look_back = 20 # Usaremos 20 passos de tempo para prever o 21º
X_train, y_train = create_sequences(train_data, look_back)
X_test, y_test = create_sequences(test_data, look_back)

# As RNNs no Keras esperam a entrada no formato: [amostras, passos_de_tempo, features]
# No nosso caso, o número de features é 1.
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))

print(f"Formato de X_train: {X_train.shape}") # (amostras, timesteps, features)
print(f"Formato de y_train: {y_train.shape}")

In [ ]:
"""
### Modelo 1: SimpleRNN

Nossa primeira rede terá uma camada `SimpleRNN` e uma camada `Dense` de saída.
"""
tf.random.set_seed(42) # para reprodutibilidade

rnn_model = Sequential()
rnn_model.add(SimpleRNN(units=50, activation='relu', input_shape=(look_back, 1)))
rnn_model.add(Dense(units=1))

rnn_model.compile(optimizer='adam', loss='mean_squared_error')
rnn_model.summary()

# Treinando o modelo
print("\nTreinando a SimpleRNN...")
history_rnn = rnn_model.fit(X_train, y_train, epochs=25, batch_size=32, verbose=1)

In [ ]:
"""
### Modelo 2: LSTM (Long Short-Term Memory)

A LSTM é uma evolução da RNN que usa "portões" (gates) para controlar o fluxo de informação,
o que a ajuda a capturar dependências de longo prazo e evitar o problema de desaparecimento do gradiente.
A estrutura do modelo é a mesma, apenas trocamos a camada.
"""
tf.random.set_seed(42)

lstm_model = Sequential()
lstm_model.add(LSTM(units=50, activation='relu', input_shape=(look_back, 1)))
lstm_model.add(Dense(units=1))

lstm_model.compile(optimizer='adam', loss='mean_squared_error')
lstm_model.summary()

# Treinando o modelo
print("\nTreinando a LSTM...")
history_lstm = lstm_model.fit(X_train, y_train, epochs=25, batch_size=32, verbose=1)

In [ ]:
# Fazendo previsões com ambos os modelos
train_predict_rnn = rnn_model.predict(X_train)
test_predict_rnn = rnn_model.predict(X_test)

train_predict_lstm = lstm_model.predict(X_train)
test_predict_lstm = lstm_model.predict(X_test)

# Invertendo a normalização para a escala original
train_predict_rnn = scaler.inverse_transform(train_predict_rnn)
test_predict_rnn = scaler.inverse_transform(test_predict_rnn)
train_predict_lstm = scaler.inverse_transform(train_predict_lstm)
test_predict_lstm = scaler.inverse_transform(test_predict_lstm)
y_test_inv = scaler.inverse_transform(y_test.reshape(-1, 1))

# Calculando o erro
rmse_rnn = np.sqrt(mean_squared_error(y_test_inv, test_predict_rnn))
rmse_lstm = np.sqrt(mean_squared_error(y_test_inv, test_predict_lstm))
print(f"\nRMSE da SimpleRNN no teste: {rmse_rnn:.4f}")
print(f"RMSE da LSTM no teste:    {rmse_lstm:.4f}")


In [ ]:
"""
### Visualizando as Previsões

Vamos plotar os dados originais e as previsões dos nossos modelos para ver como eles se saíram.
A linha azul representa os dados reais, enquanto as outras linhas mostram as previsões.
"""
# Criando um array vazio para plotar as previsões na posição correta do eixo X
plot_data = np.empty_like(data_scaled)
plot_data[:, :] = np.nan

# Previsões da RNN
plot_data[len(train_data)+look_back+1:len(data_scaled)-1, :] = test_predict_rnn

# Previsões da LSTM
plot_data_lstm = np.empty_like(data_scaled)
plot_data_lstm[:, :] = np.nan
plot_data_lstm[len(train_data)+look_back+1:len(data_scaled)-1, :] = test_predict_lstm


plt.figure(figsize=(16, 7))
plt.plot(scaler.inverse_transform(data_scaled), label="Dados Originais", color='blue')
plt.plot(scaler.inverse_transform(plot_data), label="Previsão SimpleRNN", color='orange', linestyle='--')
plt.plot(scaler.inverse_transform(plot_data_lstm), label="Previsão LSTM", color='red', linestyle='--')
plt.title("Comparação: Dados Originais vs. Previsões RNN/LSTM")
plt.xlabel("Tempo")
plt.ylabel("Amplitude")
plt.legend()
plt.grid(True)
plt.show()